In [1]:
import pandas as pd

In [2]:
columns = ['userid', 'movieid', 'rating', 'timestamp']
ratings = pd.read_table("ml-1m/ratings.dat", names = columns, sep = "::", encoding = "latin1", engine='python')
ratings

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
# columns = ["movieid", "title", "genre"]
# movies = pd.read_table("ml-1m/movies.dat", names = columns, sep = "::", encoding = "latin1", engine = "python")
# movies

In [4]:
# columns = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
# users = pd.read_table("ml-1m/users.dat", names = columns, sep = "::", encoding = "latin1", engine='python')
# users

In [9]:
from python_splitters import python_stratified_split

# Now you can use the module as expected
train_df, test_df = python_stratified_split(ratings, ratio=0.75)

In [10]:
train_df

,userid,movieid,rating,timestamp
10,1,595,5,978824268
42,1,1962,4,978301753
29,1,745,3,978824268
6,1,1287,5,978302039
32,1,1566,4,978824330
...,...,...,...,...
1000135,6040,1374,4,956715714
999921,6040,1834,4,956705073
999976,6040,1587,1,956716374
1000093,6040,3201,4,957716904


In [11]:
test_df

,userid,movieid,rating,timestamp
2,1,914,3,978301968
51,1,608,4,978301398
25,1,48,5,978824351
35,1,783,4,978824291
12,1,2398,4,978302281
...,...,...,...,...
1000118,6040,457,4,997454349
1000023,6040,45,3,956704953
999920,6040,213,5,956704056
999902,6040,2421,1,956716588


In [6]:
# import torch
# from torch.utils.data import Dataset, DataLoader, Subset
# from sklearn.model_selection import train_test_split
# import pandas as pd

# class MovieLensDataset(Dataset):
#     def __init__(self, root_folder):
#         ratings_columns = ['userid', 'movieid', 'rating', 'timestamp']
#         self.ratings = pd.read_table(root_folder + "ratings.dat", names=ratings_columns, sep="::", encoding="latin1", engine='python')
#         self.ratings = self.ratings.drop("timestamp", axis=1)

#         movies_columns = ["movieid", "title", "genre"]
#         self.movies = pd.read_table(root_folder + "movies.dat", names=movies_columns, sep="::", encoding="latin1", engine="python")

#         user_columns = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
#         self.users = pd.read_table(root_folder + "users.dat", names=user_columns, sep="::", encoding="latin1", engine='python')

#     def __len__(self):
#         return len(self.ratings)
    
#     def __getitem__(self, idx):
#         row = self.ratings.iloc[idx]
#         return torch.tensor(row.userid), torch.tensor(row.movieid), torch.tensor(row.rating)

# # Load the dataset
# dataset = MovieLensDataset("ml-1m/")

# # Split indices for train and test sets
# train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)

# # Create training and testing datasets using Subset
# train_dataset = Subset(dataset, train_indices)
# test_dataset = Subset(dataset, test_indices)

# # Example: create DataLoaders for train and test sets
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# # Iterate through a batch of the train loader for demonstration
# for user_id, movie_id, rating in train_loader:
#     print(user_id, movie_id, rating)
#     break

In [7]:
import torch
import torch.nn as nn

class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, num_layers):
        super(LightGCN, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        
        # Embeddings for users and items
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Initialize embeddings
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)
        
    def forward(self, edge_index):
        user_embeddings = self.user_embedding.weight
        item_embeddings = self.item_embedding.weight

        # Concatenate user and item embeddings
        all_embeddings = torch.cat([user_embeddings, item_embeddings])
        
        final_embeddings = all_embeddings
        
        # Graph convolution (LightGCN layers)
        for _ in range(self.num_layers):
            final_embeddings = self.propagate(edge_index, final_embeddings)
        
        user_final, item_final = torch.split(final_embeddings, [self.num_users, self.num_items])
        return user_final, item_final
    
    def propagate(self, edge_index, embeddings):
        # Simplified propagation (average over neighbors)
        row, col = edge_index
        aggregated = torch.zeros_like(embeddings)
        count = torch.zeros(embeddings.shape[0], device=embeddings.device)
        aggregated.index_add_(0, col, embeddings[row])
        count.index_add_(0, col, torch.ones_like(row, dtype=torch.float32))
        count = torch.clamp(count, min=1.0)  # Prevent division by zero
        aggregated /= count.view(-1, 1)
        return aggregated

ModuleNotFoundError: No module named 'torch'

In [ ]:
num_users = ratings['userid'].nunique()
num_items = ratings['movieid'].nunique()
embedding_dim = 64
num_layers = 2
model = LightGCN(num_users, num_items, embedding_dim, num_layers)
print(model)

: 

In [ ]:
import torch.optim as optim

LR = 0.0001
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.BCELoss()

: 